### H_Games

Delete H_Game

In [25]:
import pandas as pd
from datetime import datetime
import sqlite3
import hashlib
import ssl
from urllib.error import HTTPError
ssl._create_default_https_context = ssl._create_unverified_context

In [26]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
table_name='H_Game'
connection=sqlite3.connect(database=data_base)
cursor = connection.cursor()
drop_query = f"DROP TABLE IF EXISTS {table_name};"
cursor.execute(drop_query)
connection.commit()
connection.close()

Prepare Source from 'D:/AFL_GRP/DataBase/AFLStats.db'.  This is done via 'Load_All_Games_Plyed.

In [27]:
data_base=f'D:/AFL_GRP/DataBase/AFLStats.db'
connection=sqlite3.connect(data_base)
query=f''' 
        select Year
        ,Round
        ,Team
        ,Opponent 
        from GamesPlayed 
        where 1=1
       
    '''
df_game=pd.read_sql(query,connection)
connection.close()
df_game


,Year,Round,Team,Opponent
0,1897,R01,BL,CA
1,1897,R01,CW,SK
2,1897,R01,GE,ES
3,1897,R01,SW,ME
4,1897,R02,SW,CA
...,...,...,...,...
16617,2024,SF,PA,HW
16618,2024,SF,GW,BL
16619,2024,PF,SW,PA
16620,2024,PF,GE,BL


In [28]:
def hash_row(row):
    combined = str(row['Year']) + str(row['Round']) + str(row['Team']) + str(row['Opponent'])
    # Generate a hash using SHA-256
    return hashlib.sha256(combined.encode()).hexdigest()

df_game['Game_Hashkey']=df_game['Game_Hashkey'] = df_game.apply(hash_row, axis=1)
df_game['Load_Date']=datetime.now().date()
df_game['Record_Source']='GamesPlayed'

df_game=df_game[['Game_Hashkey','Year','Round','Team','Opponent','Load_Date','Record_Source']]
df_game

,Game_Hashkey,Year,Round,Team,Opponent,Load_Date,Record_Source
0,940baad6e3b1f642ebd20ef7f2f08033a4c38ec9633a4e...,1897,R01,BL,CA,2024-10-13,GamesPlayed
1,596615ac0b4e8508c5cbf35782f6205f35b1fe87592f40...,1897,R01,CW,SK,2024-10-13,GamesPlayed
2,a79f663d1f61f8909f15adebce2ff417fbe205cde73112...,1897,R01,GE,ES,2024-10-13,GamesPlayed
3,94c731f7c2d3e4e99889d206d252c5492c679117c8bda1...,1897,R01,SW,ME,2024-10-13,GamesPlayed
4,c5b1319e8e8bdc2f7024068131b81857811b5768cbb777...,1897,R02,SW,CA,2024-10-13,GamesPlayed
...,...,...,...,...,...,...,...
16617,4e36b57e6c268c3dea3e52cbaa32d906ec9d44b1e14886...,2024,SF,PA,HW,2024-10-13,GamesPlayed
16618,24f70b6581fd27418323e1e9a6d176ed1df6f4e3cf9b65...,2024,SF,GW,BL,2024-10-13,GamesPlayed
16619,ec692efb8fa838799813568da7c9014f816ef21679fdfc...,2024,PF,SW,PA,2024-10-13,GamesPlayed
16620,2da19cdc91d7ddf46e58c77f4e17624878c3ccf6a0ed52...,2024,PF,GE,BL,2024-10-13,GamesPlayed


In [29]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
table_name='H_Game'
connection=sqlite3.connect(database=data_base)
df_game.to_sql(table_name, connection, index=False, if_exists='append')
connection.close()

In [30]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
connection=sqlite3.connect(database=data_base)
query = f'''
with duplicate_games as
(Select  Game_hashkey
from h_game
group by Game_hashkey 
having count(1)>1)

select * from h_game where game_hashkey in (select game_hashkey from duplicate_games)
'''
df=pd.read_sql(query,connection)
connection.close()
df

,Game_Hashkey,Year,Round,Team,Opponent,Load_Date,Record_Source
0,0726c9599ffd4e9085f54825ccf8c8ef1162cf3560469b...,1928,SF,CW,ME,2024-10-13,GamesPlayed
1,0726c9599ffd4e9085f54825ccf8c8ef1162cf3560469b...,1928,SF,CW,ME,2024-10-13,GamesPlayed
2,88c993ff2dd1fe0a81bbb0a18b288f14d8509f32f34938...,1946,SF,ES,CW,2024-10-13,GamesPlayed
3,88c993ff2dd1fe0a81bbb0a18b288f14d8509f32f34938...,1946,SF,ES,CW,2024-10-13,GamesPlayed
4,7a8963d08171bc1876a1b3d0e5d684143b9638502dac1a...,1948,GF,ES,ME,2024-10-13,GamesPlayed
5,7a8963d08171bc1876a1b3d0e5d684143b9638502dac1a...,1948,GF,ES,ME,2024-10-13,GamesPlayed
6,de496d1fb1982b4594429d44bee0b610639d868c962130...,1962,PF,GE,CA,2024-10-13,GamesPlayed
7,de496d1fb1982b4594429d44bee0b610639d868c962130...,1962,PF,GE,CA,2024-10-13,GamesPlayed
8,008e5e637bfd4fc437dc78bd6bb6be2ce48726878e18e6...,1972,SF,CA,RI,2024-10-13,GamesPlayed
9,008e5e637bfd4fc437dc78bd6bb6be2ce48726878e18e6...,1972,SF,CA,RI,2024-10-13,GamesPlayed


This due to drwan game , at the moment only fix, the SK,CW 2010 grand final